<a href="https://colab.research.google.com/github/finiteautomata/rioplatense_hate_speech/blob/main/notebooks/Mixtral_Colab_(run_only_in_A100).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pysentimiento accelerate bitsandbytes datasets



In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = "mistralai/Mixtral-8x7B-Instruct-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_id)

quantization_config = BitsAndBytesConfig(
    #load_in_8bit=True
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16
)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=quantization_config)

prompt = "[INST] Explain what a Mixture of Experts is in less than 100 words. [/INST]"
inputs = tokenizer(prompt, return_tensors="pt").to(0)

output = model.generate(**inputs, max_new_tokens=50)
print(tokenizer.decode(output[0], skip_special_tokens=True))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/19 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[INST] Explain what a Mixture of Experts is in less than 100 words. [/INST] A Mixture of Experts is an ensemble learning model that combines multiple simple models, or "experts," to make accurate predictions. Each expert specializes in different parts of the input space, and a gating network determines the appropriate expert to


In [15]:
prompt = """[INST] Determinar si el siguiente mensaje contiene discurso de odio.

Entendemos que hay discurso de odio en el tweet si tiene declaraciones de carácter intenso e irracional de rechazo, enemistad y aborrecimiento contra un individuo o contra un grupo, siendo objetivos de estas expresiones por poseer una característica protegida. Las características protegidas que contemplamos son:

- mujer: mujeres o movimiento feminista
- lgbti: contra gays, lesbianas, transexuales y otras identidades de género
- racismo: inmigrantes, xenofobia, o contra pueblos aborígenes
- clase: personas de bajos recursos o motivos de clase
- discapacidad: discapacidad, problemas de salud mental o de adicción al alcohol u otras drogas
- política: ideología política
- aspecto: aspecto, gordofobia o edad
- criminal: presos o delincuentes comunes

Responder una o varias de las características separadas por coma, o bien "nada" si no hay discurso de odio. Pensar paso a paso la respuesta antes de responder.
[/INST]

contexto: Al borde del llanto, Nati Jota pidió disculpas por sus repudiables tuits pero los justificó: 'Quería sumar seguidores'
texto: Show o nada mi amorrr!! Sos una imbesil!
salida: Es un insulto y no menciona nada relacionado a las características protegidas. La respuesta final es 'nada'
---
contexto: Violento motín en la cárcel de Devoto: tomaron e incendiaron pabellones
texto: Hermanito lo que son esos grones. Ahí adentro le robaron la billetera al coronavirus
salida: La palabra "grones" significa negros, lo cual alude a su color de piel y aspecto. Además, se refiere a su condición de presos. La respuesta final es "criminal, racismo, aspecto"
---
contexto: Mayra Mendoza criticó el banderazo contra el Gobierno: "No tiene sentido y es antidemocrático"
texto: Que negrita villera....ni con la plata q roba puede arreglarse
salida: El texto alude a su color de piel, a su condición social (villera), y a su aspecto. La respuesta final es "racismo, aspecto, clase"
---
contexto: Les darán DNI provisorio a personas en situación vulnerable, para que puedan empezar a cobrar planes sociales
texto: Seguimos alimentando vagos
salida: El comentario se refiere a quienes cobran planes sociales como vagos. La respuesta final es "clase"
---
contexto: Viola Davis, Halle Berry y Angela Bassett, actrices negras que superaron la discriminación en Hollywood
texto: el titulo de por si da pena, la nación cree que el racismo es una enfermedad que los que la sufren tienen que superar, es un título discriminatorio
salida: El comentario critica el título del artículo por insinuar que el racismo es una enfermedad. No critica en ningún momento a las actrices. La respuesta final es "nada"
---
contexto: Otro ataque de furia contra un colectivero: una mujer trans le pateó la unidad y le tiró un ladrillazo
texto: Un tipo operado. Con la fuerza de un hombre y no la de una mujer
salida: El texto alude a que la mujer trans es un hombre. La respuesta final es "lgbti"
---
contexto: Elisa Carrió denunció que el Gobierno usa la pandemia para "establecer un estado de sitio"
texto: Gorda psiquiátrica
salida: El texto alude a su aspecto (gorda) y la acusa de tener problemas psiquiátricos. La respuesta final es "aspecto, discapacidad"
---
contexto: 'País de maricas': ¿Por qué Jair Bolsonaro vuelve a atacar a los homosexuales en Brasil?
texto: El discurso de bolsonaro donde dice "maricas" es brillante, sólo eso basta para meresca ser el presidente de Sudamérica. Clarín es una cueva de zurdos y progresistas que se pintan los labios punto. Clarín suavecito, te pasas de marica!
salida: El texto alude a la homosexualidad como algo negativo, a la vez que acusa de ser "zurdos y progresistas" a quienes no están de acuerdo con el discurso de Bolsonaro. La respuesta final es "lgbti, política"
---
contexto: Los dos presos heridos de bala en el motín de Devoto tienen Covid-19 y uno quedó hemipléjico
texto: justicia divina!
salida: El texto alude a que los presos merecen ser baleados. La respuesta final es 'criminal'
</s>
[INST]
contexto: {contexto}
texto: {texto}
[/INST]
"""




In [16]:
from datasets import load_dataset

dataset_name = "piuba-bigdata/contextualized_hate_speech"

ds = load_dataset(dataset_name)

In [17]:
test_ds = ds["test"].shuffle()

In [18]:
from pysentimiento.preprocessing import preprocess_tweet
import re

url_regex = r"\burl\b"


def text_preprocess(text):
    text = preprocess_tweet(text, preprocess_hashtags=False, demoji=False)
    text = text.replace("@usuario", "")

    text = re.sub(url_regex, "", text)
    # Replace multiple spaces with one
    text = " ".join(text.split())
    text = text.replace("\n", " ")

    return text


In [48]:
tokenizer.model_max_length = 6400
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
from tqdm.auto import tqdm

outs = {}

prompts = []
for example in tqdm(test_ds):
    texto = text_preprocess(example["text"])

    contexto = preprocess_tweet(
        example["context_tweet"],
        preprocess_hashtags=False,
        demoji=False,
        preprocess_handles=False,
    )
    model_input = prompt.format(contexto=contexto, texto=texto)

    prompts.append((example["id"], model_input))

batch_size = 8

pbar = tqdm(total=len(test_ds))
for i in range(0, len(test_ds), batch_size):

    ids, inputs = zip(*prompts[i:i+batch_size])
    inputs = tokenizer(inputs, return_tensors="pt", truncation=True, padding=True).to(0)

    output = model.generate(**inputs, max_new_tokens=150)

    for k in range(batch_size):
        id = ids[k]

        output_text = tokenizer.decode(output[k], skip_special_tokens=True)
        outs[id] = output_text

    pbar.update(batch_size)
    #for _ in range(4):
    #    print("="*80)
    #print(f"contexto: {contexto}")
    #print(f"texto: {texto}")
    #print(output_text)


  0%|          | 0/11343 [00:00<?, ?it/s]

  0%|          | 0/11343 [00:00<?, ?it/s]